In [1]:
# load data
import pickle

with open("piano_data.pkl", "rb") as f:
    data = pickle.load(f)
    print('X:',len(data))

329


In [2]:
data[0][0].keys()

dict_keys(['start_time', 'end_time', 'pitch', 'velocity', 'mfccs'])

In [3]:
filtered_data = []
for song in data:
    filtered_song = []
    for note in song:
        if (note['mfccs'][0].sum()!=0) and (note['mfccs'].shape[1] <= 50):
            filtered_song.append(note)
    filtered_data.append(filtered_song)

In [6]:
# make X (MFCCs) and y (note lables):
import numpy as np

X = []
for song in filtered_data:
    for note in song:
        X.append(note['mfccs'])

y = []
for song in filtered_data:
    for note in song:
        note_servo = [note['pitch'], note['velocity']]
        y.append(np.array(note_servo))
y = np.array(y)

In [7]:
note_len_store = []
for note in X:
    note_len_store.append(len(note[0]))
max_note_length = max(note_len_store)
max_note_length

for i, note in enumerate(X):
    X[i] = np.pad(note, ((0, 0), (0, max_note_length - note.shape[1])), mode='constant')

X = np.array(X)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
for i, note in enumerate(X):
    X[i] = scaler.fit_transform(note) # transform the whole thing

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the CNN model
model = models.Sequential()

# Input layer
model.add(layers.Input(shape=(13, 50)))  # 1754 time steps for each MFCC

# Convolutional layers
model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=2))

# Flatten layer
model.add(layers.Flatten())

# Fully connected layers
model.add(layers.Dense(units=128, activation='relu'))
model.add(layers.Dropout(0.5))  # Optional dropout for regularization
model.add(layers.Dense(units=64, activation='relu'))

# Output layer
# Assuming you want to predict MIDI note information (pitch, velocity)
output_dim = 2  # Adjust based on your output dimensions
model.add(layers.Dense(units=output_dim, activation='linear'))  # Linear activation for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')  # Use appropriate loss function

# Summary of the model architecture
model.summary()





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 11, 32)            4832      
                                                                 
 max_pooling1d (MaxPooling1  (None, 5, 32)             0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 3, 64)             6208      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 1, 64)             0         
 g1D)                                                            
                                                                 
 flatten (Flatten)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 128)            

In [12]:
results = model.fit(X_train, y_train, 
                    epochs=5, batch_size=64,
                    validation_data=(X_test, y_test))

Epoch 1/5

8414/8414 [==============================] - 30s 3ms/step - loss: 171.8621 - val_loss: 994.3377
Epoch 2/5
8414/8414 [==============================] - 29s 3ms/step - loss: 132.1726 - val_loss: 903.5503
Epoch 3/5
8414/8414 [==============================] - 29s 3ms/step - loss: 127.4046 - val_loss: 743.9528
Epoch 4/5
8414/8414 [==============================] - 27s 3ms/step - loss: 124.8515 - val_loss: 618.1443
Epoch 5/5
8414/8414 [==============================] - 28s 3ms/step - loss: 123.0028 - val_loss: 523.1492
